#문제-1
다음 데이터프레임의 `age` 컬럼에는 177개의 결측값이 존재합니다. `pclass`와 `sex`별로 평균을 구해서 결측값을 채워봅시다. 예를 들어, `pclass`가 1인 남성의 평균을 구해서 `pclass`가 1인 남성의 결측값을 채우세요.

In [ ]:
import pandas as pd
df = pd.read_csv("https://drive.google.com/u/0/uc?id=1ouQckC0yl-gxUl8i3wVtXvIcLUWboA0B&export=download")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


# 문제-2
`get_purchase_data` 함수는 지역코드(`code`)와 날짜정보(`date`)를 입력받아 아파트 실거래 가격을 조회합니다. 
- 지역코드 41133은 성남입니다.
- 날짜는 `MMDD` 형태로 입력해야 합니다. 예: `202101`

참고로 서버가 제공하지 않는 값은 `NaN`으로 표기됩니다.

In [1]:
!pip install xmltodict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import datetime
import pandas as pd
import xmltodict
import numpy as np

#### 공공데이터 가입하고 api 받아야함 https://www.data.go.kr/
# 활용신청 : https://www.data.go.kr/data/15058747/openapi.do
# 활용신청2 : https://www.data.go.kr/data/15058017/openapi.do

def get_purchase_data(code, date):
    url ="http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?"
    service_key = "#####################################"    

    # 4161011100
    payload = "LAWD_CD=" + code + "&" + \
              "DEAL_YMD=" + date + "&" + \
              "serviceKey=" + service_key + "&" 

    res = requests.get(url + payload)    
    xml = xmltodict.parse(res.text)    
    
    if xml['response']['body']['items'] != None and isinstance(xml['response']['body']['items']['item'], list):
        return pd.DataFrame(xml['response']['body']['items']['item']).fillna(value=np.nan)
    else:
        return pd.DataFrame()


df = get_purchase_data("41133", "202101")
df.head()

KeyError: ignored

메모리를 효과적으로 사용하기 위해 데이터 타입을 정리해 봅시다. 
- 숫자, category, Timestamp로 정리해 보세요.

In [ ]:
df.info()

# 문제-3
`get_rent_data` 함수는 지역코드(`code`)와 날짜정보(`date`)를 입력받아 아파트 전월세 가격을 조회합니다. 
- 지역코드 41133은 성남입니다.
- 날짜는 `MMDD` 형태로 입력해야 합니다. 예: `202101`

In [ ]:
def get_rent_data(code, date):    
    service_key = "#####################################"    
    url = f"http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptRent?serviceKey={service_key}&LAWD_CD={code}&DEAL_YMD={date}"
    res = requests.get(url)    
    xml = xmltodict.parse(res.text)
    if xml['response']['body']['items']:
        return pd.DataFrame(xml['response']['body']['items']['item']).fillna(value=np.nan)
    else:
        return pd.DataFrame()

df_r = get_rent_data("41133", "202101")
df_r.head()

메모리를 효과적으로 사용하기 위해 데이터 타입을 정리해 봅시다. 
- 숫자, category, Timestamp로 정리해 보세요.

# 문제-4
`get_purchase_data` 함수와 `get_rent_data` 함수를 사용해서 성남의 `2021`년도(1년간) 데이터를 수집해 봅시다. `df_m`에는 1년간의 매매데이터를 `df_j`에는 1년간의 전세데이터를 저장하세요.


# 문제-5
문제-4에서 수집한 데이터로 성남의 `2021`년도(1년간) 월별 전세가율의 평균을 시각화해 봅시다. 전세가율은 다음의 수식으로 계산합니다.   
- $ 전세가율 = \tfrac {매매가격} {전세가격} $

데이터는 `FacetGrid`로 시각화하며, `x`축에는 시간 `y`축에는 전세가율을 그립니다. 이 때, `법정동` 별로 데이터를 나눠서 시각화하세요.

In [ ]:
df_r.법정동.unique()